# DSPy Concise COT

In [1]:
#|default_exp dspy.predict.ccot

In [2]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [3]:
#|export

import dspy
from dspy.primitives.program import Module
from dspy.signatures.signature import ensure_signature

from bellem.logging import get_logger

log = get_logger(__name__)

In [5]:
#|export

class ConciseChainOfThought(Module):
    def __init__(self, signature, rationale_type=None, activated=True, **config):
        super().__init__()

        self.activated = activated

        self.signature = signature = ensure_signature(signature)
        *_keys, last_key = signature.output_fields.keys()

        prefix = "Reasoning: Let's think step by step in order to"

        if isinstance(dspy.settings.lm, dspy.LM):
            desc = "${reasoning}"
        elif dspy.settings.experimental:
            desc = "${produce the output fields}. Be concise. We ..."
        else:
            desc = f"${{produce the {last_key}}}. Be concise. We ..."

        rationale_type = rationale_type or dspy.OutputField(prefix=prefix, desc=desc)

        # Add "rationale" field to the output signature.
        if isinstance(dspy.settings.lm, dspy.LM) or dspy.settings.experimental:
            extended_signature = signature.prepend("reasoning", rationale_type, type_=str)
        else:
            extended_signature = signature.prepend("rationale", rationale_type, type_=str)
        
        self._predict = dspy.Predict(extended_signature, **config)
        self._predict.extended_signature = extended_signature

    def forward(self, **kwargs):
        assert self.activated in [True, False]

        signature = kwargs.pop("new_signature", self._predict.extended_signature if self.activated else self.signature)
        return self._predict(signature=signature, **kwargs)

    @property
    def demos(self):
        return self._predict.demos

    @property
    def extended_signature(self):
        return self._predict.extended_signature

In [6]:
#|hide
import nbdev; nbdev.nbdev_export()